# Zielstellung

- Mithilfe des Codes aus dieser Datei sollen verschiedene generelle Raumanalysen möglich gemacht werden
- dabei werden für bestimmte Punkte (Geokoordinaten) bestimmte Werte ermittelt, die anschließend auf einer heatmap farblich dargestellt werden


# Umwandeln der Koordinaten in ein besser nutzbares Format

- Die Geokoordinateni n WSG84 werden in das für Deutschland geeignete epsg:31468-Format umgewandelt
- von nun an können sie wie Angaben in Metern betrachtet werden, insbesondere die Abstandsberechnung wird dadurch deutlich vereinfacht

In [8]:

import json
from tqdm import tqdm

def extract_coordinates():
    from pyproj import Proj
    data = {}
    with open("gtfsmdvlvb/stops.txt", "r") as f:
        txt = [elem.split(",") for elem in f.read().split("\n")[1:-1]]
    for elem in tqdm(txt):
        #data[elem[0]] = [float(elem[-2]), float(elem[-1])]
        data[elem[0]] = Proj('epsg:31468')(float(elem[-1]), float(elem[-2]))
    with open("lvb_auswertung/coordinates.json", "w+") as f:
        json.dump(data, f, indent=4)
    return data

def load_coordinates():
    with open("lvb_auswertung/coordinates.json", "r") as f:
        return json.load(f)

def min_max():
    data = load_coordinates()
    x = [data[elem][0] for elem in data]
    y = [data[elem][1] for elem in data]
    return [(min(x), max(x)), (min(y), max(y))]

# gibt Zwischenschritte zwischen Extremkoordinaten abhängig von einer Distanz für ein grid zurück
def steps(dist):
    m_m = min_max()
    cur_x = m_m[0][0]
    cur_y = m_m[1][0]
    x_steps, y_steps = [cur_x], [cur_y]
    while cur_x < m_m[0][1]:
        cur_x += dist
        x_steps.append(cur_x)
    while cur_y < m_m[1][1]:
        cur_y += dist
        y_steps.append(cur_y)
    return [x_steps, y_steps]


def step_coords(dist):
    steps = steps(dist)
    
#extract_coordinates()
#print(min_max())
c = steps(500)
print(len(c[0])*len(c[1]))

48705


# Extrahieren von Frequenzen an einer Haltestelle

- für jede Haltestelle wird anhand der Routen in stop_times ermittelt, wie oft dort ein Verkehrsmittel hält

Vereinfachungen:
- jedes Verkehrsmittel gleich behandelt (beispielsweise nicht gewichtet anhand der Anzahl an Menschen, die in es passen würden)
- Annahme, dass Verkehrsmittel über Tag hinweg, jeden Tag gleich oft kommen
  - keine gravierende Vereinfachung, da wir untersuchen, welche Regionen gut ausgebaut sind, Anzahl Verkehrsmittel pro Tag scheint guter Vergleichsindikator

In [17]:
import json
from utils import load_stops, load_stop_times
from tqdm import tqdm

def calculate_frequencies():
    stop_data = [stop[0] for stop in load_stops()]
    trip_data = [stop_time[3] for stop_time in load_stop_times()]
    #data = load_coordinates()
    with open("lvb_auswertung/coordinates.json", "r") as f:
        data = json.load(f)
    for elem in tqdm(stop_data):
        if len(data[elem]) == 2:
            data[elem].append(trip_data.count(elem))
        else:
            data[elem][3] = trip_data.count(elem)
    with open("lvb_auswertung/coordinates.json", "w+") as f:
        json.dump(data, f, indent=4)
    return data

c = calculate_frequencies()
print(c["0000145"])

100%|██████████| 4377/4377 [02:51<00:00, 25.54it/s]


[4531101.507033625, 5691819.717443315, 1618]


# Finden von Haltestellendichten an einem Punkt

In [15]:
from copy import deepcopy
from tqdm import tqdm
import json

# for every point, we execute this function, that gives back a value based on the distances to all other points
# TODO: Gewichtung eines Punktes (zum Beispiel relevant für Anzahl der Verbindungen an einer Haltestelle -> könnte in die Haltestellendichte einfließen)
def all_distances_point(coordinate, haltestellen):
    distances = []
    for other_coordinate in haltestellen:
        weighting = 1 if len(other_coordinate) == 2 else other_coordinate[3-1]
        coordinate, other_coordinate = tuple(coordinate[:2]), tuple(other_coordinate[:2])
        #weighting = 1
        #GD(coordinate, other_coordinate).m
        distances.append((
                ((coordinate[0]-other_coordinate[0])**2+(coordinate[1]-other_coordinate[1])**2)**(1/2),
                weighting
        ))
        #if distances[-1][0] == 0:
        #    distances = distances[:-1]
    #while 0 in distances: distances.remove(0)
    distances = [distance for distance in distances if distance[0] != 0]
    if distances is None: return 0
    value = sum([x[1]/x[0] for ind, x in enumerate(distances)])
    return value

# Hier müssen wir nochmal überlegen, wie wir die Punkte ausdrücken wollen (idealerweise nicht durch die Koordinaten, sondern durch stop-indizes beziehungsweise Rasterkoordinaten (ganzzahlig als Tupel bzw string im format '{x}|{y}'))
def all_distances(points, haltestellen):
    distances = []
    for point in tqdm(points):
        distances.append([point, all_distances_point(points[point], deepcopy(haltestellen))])
    return distances

def haltestellendichte(weighting=False):
    with open("lvb_auswertung/coordinates.json", "r") as f:
        data = json.load(f)
        # without weighting
        if not weighting:
            coordinates = [data[elem][:2] for elem in data]
            goal_file = "dichte"
        else:
        # with weightings by stops per day
            coordinates = [data[elem] for elem in data]
            goal_file = "dichte_weighting"
    
    with open(f"lvb_auswertung/{goal_file}.json", "w+") as f:
        json.dump(all_distances(data, coordinates), f, indent=4)


haltestellendichte()
haltestellendichte(weighting=True)


100%|██████████| 4377/4377 [01:03<00:00, 69.36it/s]
